**KAIST, Fall 2020**   
**[CoE202(A)] Fundamentals of Artificial Intelligence**

**Final Project: Cancer Classifier**

**Instructor: Prof. Young-Gyu Yoon**

In [ ]:
##### External Libraries
import os
import sys
import time
import torch
import torchvision.transforms as transforms
import numpy as np
import glob
from PIL import Image

##### Mount drive
from google.colab import drive
drive.mount("/content/drive")

##### Own libraries
filepath = "/content/drive/My Drive/CoE202/proj/src"
sys.path.append(filepath)

from model_baseline import cancer_classifier

##### Load trained model
state_path = f"{filepath}/results/model_state_dict.pt"  # Model location
experiment = "results"  # Experiment name (For saving the result)

model = cancer_classifier(state_path)
print(model)
model.eval()
# If loading is done with no error, you will see  " <All keys matched successfully> "

##### Data load
class cancer_test_dataset(torch.utils.data.Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.files = glob.glob(f"{root}/*.jpg")
    
    def __len__(self):
        ans = len(self.files)
        return ans

    def __getitem__(self, idx):
        image = Image.open(f"{self.root}/{str(idx + 1)}.jpg")
        if self.transform is not None:
            image = self.transform(image)
        return image

test_dataset = cancer_test_dataset(root=f"{filepath}/test/",
                                   transform=transforms.Compose([
                                                    transforms.ToTensor()
                                                ]))
test_loader = torch.utils.data.DataLoader(test_dataset,
                                        batch_size=4,
                                        shuffle=False,
                                        drop_last=False
                                        )

##### Evaluation
prediction_index = []
accuracy_list = []

for _, (batch) in enumerate(test_loader):
    batch = batch.cuda()
    prediction = model(batch)
    _, i = prediction.max(dim=1)
    prediction_index += i.tolist()

final_index = prediction_index
print(f"Example prediction, 5 items from front : {final_index[:5]}")
print(f"""{time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())} || Test finished!""")
np.savetxt(f"{filepath}/{experiment}/test_result.csv", np.dstack((np.arange(1, len(final_index)+1), final_index))[0], "%d,%d", header="Id,Values")